# My attempt at rewriting Emilio's fine tuning code

https://github.com/nuitrcs/fine_tuning_workshop

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
%load_ext autoreload
%autoreload 2

from finetuner import fineTuner

In [ ]:
train = pd.read_csv("https://raw.githubusercontent.com/MoritzLaurer/less-annotating-with-bert-nli/refs/heads/master/data_clean/df_manifesto_protectionism_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/MoritzLaurer/less-annotating-with-bert-nli/refs/heads/master/data_clean/df_manifesto_protectionism_test.csv")

train = train[train['label'] != 0]
test = test[test['label'] != 0]

train['label'] = train['label'] - 1
test['label'] = test['label'] - 1

train['text'] = train['text_preceding'] + " " + train['text_original'] + " " + train['text_following']
test['text'] = test['text_preceding'] + " " + test['text_original'] + " " + test['text_following']

train = train[train['text'].notna()]
test = test[test['text'].notna()]

SEED = 6325
validation, temp = train_test_split(test, test_size=0.5, random_state=SEED)
test, new = train_test_split(temp, test_size=0.5, random_state=SEED)

In [ ]:
my_finetuner = fineTuner()

In [ ]:
my_finetuner.create_dataset(train, validation, test)

In [ ]:
my_finetuner.define_trainer()

In [ ]:
training_output, evaluate_output = my_finetuner.train_and_evaluate()

In [ ]:
training_output

In [ ]:
evaluate_output

In [ ]:
# all in one
# my_finetuner.create_finetuned_model_from_data(train, validation, test)

In [ ]:
predictions, predicted_classes = my_finetuner.predict(new)
new['predicted_class'] = predicted_classes

In [ ]:
pd.crosstab(new['label'], new['predicted_class'], margins=True)

In [ ]:
pd.crosstab(new['country_name'], new['predicted_class'], margins=True)

In [ ]:
# check (should output an error message and not re-fine-tune the model)
training_output, evaluate_output = my_finetuner.train_and_evaluate()